In [1]:
import pandas as pd
import time
import os
import re
import json
from google import genai
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
load_dotenv()
client = genai.Client(api_key=os.getenv("api_key"))

In [15]:
df = pd.read_excel("source data/database cbo.xlsx", sheet_name=1)

In [ ]:
for idx, row in df.iterrows():
    linha_atual = idx + 1
    print(f" Processando linha {linha_atual} de {len(df)}")

    group = row["task_group"]
    task = row["task"]
    tarefa_texto = f"{group}: {task}"
    key = row["key"]

    prompt_usuario = f"""
Você é um especialista em organização do trabalho.

A seguir, há uma descrição de tarefa associada a uma ocupação e seu ID.

id_tarefa: {key}
Tarefa: {tarefa_texto}

Classifique a tarefa com base em UMA descrição EXATA contida na lista abaixo:

Entrada de Informação - Identificar e Avaliar Informações Relevantes para o Trabalho - Estimar Características Quantificáveis de Produtos, Eventos ou Informações;
Entrada de Informação - Identificar e Avaliar Informações Relevantes para o Trabalho - Identificar Objetos, Ações ou Eventos;
Entrada de Informação - Identificar e Avaliar Informações Relevantes para o Trabalho - Inspecionar Equipamentos, Estruturas ou Materiais;
Entrada de Informação - Buscar e Receber Informações Relacionadas ao Trabalho - Obter Informações;
Entrada de Informação - Buscar e Receber Informações Relacionadas ao Trabalho - Monitorar Processos, Materiais ou Ambiente;

Interação com Outros - Administrar - Monitorar e Controlar Recursos;
Interação com Outros - Administrar - Realizar Atividades Administrativas;
Interação com Outros - Administrar - Recrutamento e Seleção de Funcionários;
Interação com Outros - Comunicar e Interagir - Auxiliar e Cuidar de Outras Pessoas;
Interação com Outros - Comunicar e Interagir - Comunicar com Pessoas Fora da Organização;
Interação com Outros - Comunicar e Interagir - Comunicar com Supervisores, Colegas ou Subordinados;
Interação com Outros - Comunicar e Interagir - Estabelecer e Manter Relacionamentos Interpessoais;
Interação com Outros - Comunicar e Interagir - Interpretar o Significado da Informação para Outros;
Interação com Outros - Comunicar e Interagir - Atuar para ou Trabalhar Diretamente com o Público;
Interação com Outros - Comunicar e Interagir - Resolver Conflitos e Negociar com Outros;
Interação com Outros - Comunicar e Interagir - Vender ou Influenciar Outros;

Coordenar, Desenvolver, Gerenciar e Aconselhar - Treinar e Ensinar Outros;
Coordenar, Desenvolver, Gerenciar e Aconselhar - Coordenar o Trabalho e as Atividades de Outros;
Coordenar, Desenvolver, Gerenciar e Aconselhar - Desenvolver e Construir Equipes;
Coordenar, Desenvolver, Gerenciar e Aconselhar - Orientar, Dirigir e Motivar Subordinados;
Coordenar, Desenvolver, Gerenciar e Aconselhar - Prestar Consultoria e Aconselhamento a Outros;

Processos Mentais - Processamento de Informação e Dados - Analisar Dados ou Informações;
Processos Mentais - Processamento de Informação e Dados - Avaliar Informações para Determinar Conformidade com Padrões;
Processos Mentais - Processamento de Informação e Dados - Julgar a Qualidade de Objetos, Serviços ou Pessoas;
Processos Mentais - Processamento de Informação e Dados - Processar Informações;
Processos Mentais - Raciocínio e Tomada de Decisão - Desenvolver Objetivos e Estratégias;
Processos Mentais - Raciocínio e Tomada de Decisão - Tomar Decisões e Resolver Problemas;
Processos Mentais - Raciocínio e Tomada de Decisão - Organizar, Planejar e Priorizar o Trabalho;
Processos Mentais - Raciocínio e Tomada de Decisão - Agendar Trabalhos e Atividades;
Processos Mentais - Raciocínio e Tomada de Decisão - Pensar Criativamente;
Processos Mentais - Raciocínio e Tomada de Decisão - Atualizar e Utilizar Conhecimentos Relevantes;

Resultado do Trabalho - Realização de Atividades Complexas e Técnicas - Documentar/Registrar Informações;
Resultado do Trabalho - Realização de Atividades Complexas e Técnicas - Projetar e Especificar Dispositivos Técnicos, Peças e Equipamentos;
Resultado do Trabalho - Realização de Atividades Complexas e Técnicas - Reparar e Manter Equipamentos Eletrônicos;
Resultado do Trabalho - Realização de Atividades Complexas e Técnicas - Reparar e Manter Equipamentos Mecânicos;
Resultado do Trabalho - Realização de Atividades Complexas e Técnicas - Trabalhar com Computadores;
Resultado do Trabalho - Realização de Atividades Físicas e Manuais - Controlar Máquinas e Processos;
Resultado do Trabalho - Realização de Atividades Físicas e Manuais - Manipular e Mover Objetos;
Resultado do Trabalho - Realização de Atividades Físicas e Manuais - Operar Veículos, Equipamentos ou Dispositivos Mecanizados;
Resultado do Trabalho - Realização de Atividades Físicas e Manuais - Realizar Atividades Físicas Gerais;

---
### *FORMATO DA RESPOSTA*

Gere um JSON com a análise para cada uma das tarefas fornecidas. A resposta deve ser *apenas o objeto JSON*, sem nenhum texto introdutório ou explicações adicionais.

[
  {{
    "key": id_tarefa,
    "activity": Classificação
  }}
]
"""

    try:
        tempo_inicio = time.time()

        response = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=prompt_usuario,
        )
        resposta = response.text
        resposta = resposta.replace("```json", "").replace("```", "")
        tempo_fim = time.time()
        duracao = tempo_fim - tempo_inicio
        print(f"{key} processado em {duracao:.2f} segundos")

        arquivo_saida = os.path.join("json/activity", f"{key}.json")
        with open(arquivo_saida, "w", encoding="utf-8") as f:
            json.dump(json.loads(resposta), f, ensure_ascii=False, indent=4)

    except Exception as e:
        print(f"Erro ao processar {key} - {linha_atual}: {e}")

In [13]:
all_json = []
arquivos_com_erro = []

pasta = "json/activity"

for nome_arquivo in os.listdir(pasta):
    if nome_arquivo.endswith(".json"):
        caminho_arquivo = os.path.join(pasta, nome_arquivo)
        try:
            with open(caminho_arquivo, "r", encoding="utf-8") as f:
                conteudo = f.read().strip()
                if not conteudo:
                    raise ValueError("Arquivo vazio")
                dados = json.loads(conteudo)
                if isinstance(dados, list):
                    all_json.extend(dados)
                else:
                    all_json.append(dados)
        except Exception as e:
            print(f"Erro no arquivo '{nome_arquivo}': {e}")
            arquivos_com_erro.append(caminho_arquivo)


for caminho in arquivos_com_erro:
    try:
        os.remove(caminho)
        print(f"Arquivo deletado: {os.path.basename(caminho)}")
    except Exception as e:
        print(f"Erro ao deletar {os.path.basename(caminho)}: {e}")


with open("json/activity.json", "w", encoding="utf-8") as f:
    json.dump(all_json, f, ensure_ascii=False, indent=2)

In [14]:
with open("json/activity.json", "r", encoding="utf-8") as f:
    data = json.load(f)

df = pd.DataFrame(data)

df['key'] = df['key'].str.replace('^id ', '', regex=True)

df.to_excel("results/activity.xlsx", index=False)